# A Colab notebook for PhosBoost phosphorylation prediction

PhosBoost is a machine learning approach that leverages protein language models and gradient boosting trees (CatBoost, specifically) to predict protein phosphorylation from experimentally derived data. PhosBoost offers improved performance when recall is prioritized while consistently providing more confident probability scores. You can find more information in the [Plant Direct manuscript](https://onlinelibrary.wiley.com/doi/10.1002/pld3.554) and in the [GitHub repository](https://github.com/eporetsky/PhosBoost).

This Colab notebook was written to allow predict phosphorylated residues in a given protein sequence. It is designed to run a CPU node and takes less then a minute to setup and run the prediction. Once everything is complete you will be prompted to download the CSV file with the phosphorylation prediction results for the S/T/Y residues. The output includes the predicted probabilities for each of the residues and the phosphorylation prediction (1 if probability > 0.5, 0 otherwise).

**NOTE:** PhosBoost was trained on the experimental plant phosphorylation data using the QPTMPlants database and has not been tested on non-plant species. See the GitHub for instructions on training your own model or either send me an email or create an GitHub issue and I will try to help!


In [ ]:
#@title Setup the environment to run PhosBoost (Run this cell once at the beginning)

import warnings
warnings.filterwarnings('ignore')

!uv pip install torch torchvision torchaudio transformers sentencepiece accelerate h5py biopython catboost --extra-index-url https://download.pytorch.org/whl/cu116 > /dev/null

# Import dependencies
from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py
import time
from Bio import SeqIO

import re
import joblib
import pandas as pd
import numpy as np

# Path variables
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# check whether GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50)
def get_T5_model():
    '''
    retrieves the model and tokenizer
    '''
    # specify the encorder-part of the model
    model_checkpoint = 'Rostlab/prot_t5_xl_half_uniref50-enc'

    # import the model
    model = T5EncoderModel.from_pretrained(model_checkpoint)

    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model

    # import tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    return model, tokenizer

# loading model
model, tokenizer = get_T5_model()

# Download the PhosBoost pickled CatBoost models to run the classification
!wget --no-verbose -O qPTMplants.ST.joblib.pkl https://github.com/eporetsky/PhosBoost/blob/main/data/models/qPTMplants.ST.joblib.pkl?raw=true
!wget --no-verbose -O qPTMplants.Y.joblib.pkl https://github.com/eporetsky/PhosBoost/blob/main/data/models/qPTMplants.Y.joblib.pkl?raw=true

Using cpu
2024-05-01 03:52:19 URL:https://raw.githubusercontent.com/eporetsky/PhosBoost/main/data/models/qPTMplants.ST.joblib.pkl [8052716/8052716] -> "qPTMplants.ST.joblib.pkl" [1]
2024-05-01 03:52:20 URL:https://raw.githubusercontent.com/eporetsky/PhosBoost/main/data/models/qPTMplants.Y.joblib.pkl [3736762/3736762] -> "qPTMplants.Y.joblib.pkl" [1]


In [7]:
#@title Change the sequence and ID and hit `Runtime` -> `Run after`

seq_id = "AT4G18710" #@param {type:"string"}
seq = "MADDKEMPAAVVDGHDQVTGHIISTTIGGKNGEPKQTISYMAERVVGTGSFGIVFQAKCLETGETVAIKKVLQDRRYKNRELQLMRVMDHPNVVCLKHCFFSTTSKDELFLNLVMEYVPESLYRVLKHYSSANQRMPLVYVKLYMYQIFRGLAYIHNVAGVCHRDLKPQNLLVDPLTHQVKICDFGSAKQLVKGEANISYICSRFYRAPELIFGATEYTTSIDIWSAGCVLAELLLGQPLFPGENAVDQLVEIIKVLGTPTREEIRCMNPHYTDFRFPQIKAHPWHKIFHKRMPPEAIDFASRLLQYSPSLRCTALEACAHPFFDELREPNARLPNGRPFPPLFNFKQEVAGSSPELVNKLIPDHIKRQLGLSFLNQSGT*" #@param {type:"string"}

In [8]:
#@title This cell calculates the PLM embeddings for the S/T/Y residues

def get_ixs_sites(name, seq, sites):
    """
    Returns
    """
    ix_list = []
    site_list = []
    for pattern in sites:
        indices = re.finditer(pattern=pattern, string=seq)
        for ix in indices:
            ix = ix.start()
            ix_list.append(ix)
            site_list.append(name+"_"+pattern+str(ix+1))

    return(site_list, ix_list)

def get_embeddings(model, tokenizer, seq, seq_id,
                   max_residues=50000, max_seq_len=20000, max_batch=1000 ):
# original: max_residues=4000, max_seq_len=1000, max_batch=100
    seq.replace("*", "").replace("\n", "").replace(" ", "")
    seq_len = len(seq)
    seqs = (' '.join(list(seq)), )

    token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(token_encoding['input_ids']).to(device)
    attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

    try:
        with torch.no_grad():
            # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
            embedding_repr = model(input_ids, attention_mask=attention_mask)
    except RuntimeError:
        print("RuntimeError during embedding for {} (L={})".format(seq_id, seq_len))

    # Get the site names and the ixs in the numpy array
    sites, ixs = get_ixs_sites(seq_id, seq, "STY")

    emb = embedding_repr.last_hidden_state[0,:seq_len]

    tmp_emb = np.hstack([
        emb.detach().cpu().numpy().squeeze()[ixs,:],
        np.tile(emb.mean(dim=0).detach().cpu().numpy().squeeze(), (len(ixs), 1))
    ])

    # Add the generated ProtT5 embeddings
    results = np.empty((0, 2048))
    results = np.vstack([results, tmp_emb])
    results = pd.DataFrame(results)
    results.index = sites
    return results

results = get_embeddings(model, tokenizer, seq, seq_id)

In [9]:
#@title This cells performs the phosphorylation prediction using the PhosBoost method

preds_df = pd.DataFrame()

# Name of the embedding file(s) located in the data/embeddings/ folder for model training
# Optional: To combine multiple embedding files comma-separate each input file name
# (For example: combining training and validation data after hyperparameter tuning)
# Load the embedding data file(s) for the model to be trained on
X = results.copy()
res_list = X.index.tolist()
X["res"] = X.index.str.split("_").str[-1].str[0]
X = X[X["res"].isin(["S", "T"])]
X = X.drop("res", axis=1)
cb_model = joblib.load("qPTMplants.ST.joblib.pkl")
preds_proba = cb_model.predict_proba(X)
preds_ST = pd.DataFrame()
preds_ST["site"] = X.index.tolist()
preds_ST["probability"]  = preds_proba[:,1]
preds_ST["prediction"]  = preds_ST["probability"].apply(lambda x: 1 if x >= 0.5 else 0)

X = results.copy()
res_list = X.index.tolist()
X["res"] = X.index.str.split("_").str[-1].str[0]
X = X[X["res"].isin(["Y"])]
X = X.drop("res", axis=1)
cb_model = joblib.load("qPTMplants.Y.joblib.pkl")
preds_proba = cb_model.predict_proba(X)
preds_Y = pd.DataFrame()
preds_Y["site"] = X.index.tolist()
preds_Y["probability"]  = preds_proba[:,1]
preds_Y["prediction"]  = preds_Y["probability"].apply(lambda x: 1 if x >= 0.5 else 0)

# Create the dataframe and download it
preds_df = pd.concat([preds_ST, preds_Y])
preds_df.to_csv("{}.phosboost.csv".format(seq_id), index=False)
from google.colab import files
files.download("{}.phosboost.csv".format(seq_id))

print("PhosBoost prediction was completed for {}".format(seq_id))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PhosBoost prediction was completed for AT4G18710
